In [4]:
#Instalar bibliotecas pysark e findspark
!pip -q install pyspark findspark

In [1]:
import sys
sys.path.append('/var/lang/lib/python37.zip')
sys.path.append('/var/lang/lib/python3.7')
sys.path.append('/var/lang/lib/python3.7/lib-dynload')
sys.path.append('/var/lang/lib/python3.7/site-packages')
sys.path.remove('/opt/.sagemakerinternal/conda/lib/python3.7/site-packages')

In [7]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import boto3
import awswrangler as wr
from pandas_profiling import ProfileReport

## Spark Context and Spark Session

In [8]:
print(pyspark)

<module 'pyspark' from '/var/lang/lib/python3.7/site-packages/pyspark/__init__.py'>


In [9]:
findspark.init('/var/lang/lib/python3.7/site-packages/pyspark')
sc = SparkContext('local', 'SparkContext')
spark = SparkSession.builder.appName("SparkSession").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/02 20:13:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
#Create boto3 sesesion for Athena, SageMaker and S3 connection
my_boto3_session = boto3.Session(region_name='us-east-1',
    aws_access_key_id='xxxx',
    aws_secret_access_key='xxxxx',
    aws_session_token='xxxxxx')

## Functions

In [11]:
#funcao para desagrupar os dados que estao agrupados em struct ou array
def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']
    flat_df = nested_df.select(flat_cols +
                              [F.col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [12]:
#Constant for credits_class
def job_position_const(s):
    if s == 'leadership':
        out='Management'
    elif s in ('junior', 'senior', 'specialist', 'mid-level'):
        out='Technical'
    elif s in ('', 'null', None, 'None'):
        out="Missing"
    else:
        out="UNKNOWN"
    return out
job_position_const_func = F.udf(job_position_const, StringType())

## Import data from Athena

In [13]:
query="""SELECT context.id, context.ts, event.bodyraw, event.body FROM "dev-lakehouse-stream"."gupy";"""
df_lakehouse_table = wr.athena.read_sql_query(query, database="prod-lakehouse-mirror", boto3_session=my_boto3_session)
df_lakehouse_table = df_lakehouse_table.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('missing'))
df_lakehouse_table = df_lakehouse_table.fillna(0)
df_lakehouse_table

,id,ts,bodyraw,body
0,2AiWcTjQQpxTQ1NDleYyBBjjEsN,2022-06-17T19:09:25.679Z,"{""companyName"":""Intera"",""event"":""candidate.hir...","{""date"":""2022-06-17T19:09:25.248Z"",""data"":{""ca..."
1,2AWiQkO2oa1IgloBZkGD5biTlu0,2022-06-13T14:48:47.130Z,"{""companyName"":""Intera"",""event"":""job.changed"",...","{""date"":""2022-06-13T14:46:43.910Z"",""data"":{""re..."
2,2AZEcmYaPqIc3y2zzUqEE2MWFoU,2022-06-14T12:13:08.039Z,"{""companyName"":""Intera"",""event"":""job.changed"",...","{""date"":""2022-06-14T12:13:07.230Z"",""data"":{""re..."
3,2AcrOmnSmLZnzVys5Cd9ndiECqn,2022-06-15T19:01:24.568Z,"{""companyName"":""Intera"",""event"":""job.changed"",...","{""date"":""2022-06-15T19:01:23.747Z"",""data"":{""re..."
4,2AtdCEGxc6rAyTIFHXHDUmTvgub,2022-06-21T17:31:25.381Z,"{""companyName"":""Intera"",""event"":""candidate.hir...","{""date"":""2022-06-21T17:31:24.798Z"",""data"":{""ca..."
...,...,...,...,...
145,2AcrsxmzPIl5HqnA5haCxjPhqZF,2022-06-15T19:05:24.988Z,"{""companyName"":""Intera"",""event"":""job.changed"",...","{""date"":""2022-06-15T19:05:24.786Z"",""data"":{""re..."
146,2AWjGN5wwGdcMVVYu17hnFwnvY7,2022-06-13T14:55:38.544Z,"{""companyName"":""Intera"",""event"":""job.changed"",...","{""date"":""2022-06-13T14:48:37.994Z"",""data"":{""re..."
147,2AWiEXxZM2ktxQLhDPPOPPzgktD,2022-06-13T14:47:10.606Z,"{""companyName"":""Intera"",""event"":""job.changed"",...","{""date"":""2022-06-13T14:47:10.437Z"",""data"":{""re..."
148,2AiTzut6wV5MuAj1ek0xobGojt6,2022-06-17T18:47:51.401Z,"{""companyName"":""Intera"",""event"":""application.c...","{""date"":""2022-06-17T18:47:51.078Z"",""data"":{""ca..."


In [9]:
# schema = StructType(
# [   StructField('linkedinusername', StringType(),True),
#     StructField('name', StringType(),True),
#     StructField('stack',
#         StructType([
#             StructField('backend', StringType(),True),
#             StructField('frontend', StringType(),True)
#             ])
#     ),
# ]
# )
# schema

In [13]:
# sdf_lakehouse_table = spark.createDataFrame(linkedincompanies, schema)
sdf_lakehouse_table = spark.createDataFrame(df_lakehouse_table)
sdf_lakehouse_table.printSchema()

root
 |-- id: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- bodyraw: string (nullable = true)
 |-- body: string (nullable = true)



In [14]:
# sdf_lakehouse_table.show(truncate=False)

In [15]:
# sdf_lakehouse_table2.select('id', 'bodyraw_json.data.id', 'bodyraw_json.data.name', 'bodyraw_json.data.status', 'bodyraw_json.data.description').show(truncate=False)

In [16]:
json_schema = spark.read.json(sdf_lakehouse_table.select('bodyraw').rdd.map(lambda row: row.bodyraw)).schema
sdf_lakehouse_table2 = sdf_lakehouse_table.withColumn('bodyraw_json', F.from_json(F.col('bodyraw'), json_schema))
sdf_lakehouse_table2.printSchema()

22/06/24 00:54:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
root
 |-- id: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- bodyraw: string (nullable = true)
 |-- body: string (nullable = true)
 |-- bodyraw_json: struct (nullable = true)
 |    |-- _corrupt_record: string (nullable = true)
 |    |-- companyName: string (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- additionalInformation: string (nullable = true)
 |    |    |-- addressCity: string (nullable = true)
 |    |    |-- addressCountry: string (nullable = true)
 |    |    |-- addressCountryShortName: string (nullable = true)
 |    |    |-- addressLatitude: double (nullable = true)
 |    |    |-- addressLine: string (nullable = true)
 |    |    |-- addressLongitude: double (nullable = true)
 |    |    |-- addressNumber: string (nullable = true)
 |    |    |-- addressState

In [17]:
json_body_schema = spark.read.json(sdf_lakehouse_table.select('body').rdd.map(lambda row: row.body)).schema
sdf_lakehouse_table3 = sdf_lakehouse_table2.withColumn('body_json', F.from_json(F.col('body'), json_body_schema))
sdf_lakehouse_table3.printSchema()

root
 |-- id: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- bodyraw: string (nullable = true)
 |-- body: string (nullable = true)
 |-- bodyraw_json: struct (nullable = true)
 |    |-- _corrupt_record: string (nullable = true)
 |    |-- companyName: string (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- additionalInformation: string (nullable = true)
 |    |    |-- addressCity: string (nullable = true)
 |    |    |-- addressCountry: string (nullable = true)
 |    |    |-- addressCountryShortName: string (nullable = true)
 |    |    |-- addressLatitude: double (nullable = true)
 |    |    |-- addressLine: string (nullable = true)
 |    |    |-- addressLongitude: double (nullable = true)
 |    |    |-- addressNumber: string (nullable = true)
 |    |    |-- addressState: string (nullable = true)
 |    |    |-- addressStateShortName: string (nullable = true)
 |    |    |-- addressStreet: string (nullable = true)
 |    |    |-- addressZipCode: stri

In [18]:
sdf_bodyraw = flatten_df((flatten_df(flatten_df(flatten_df(sdf_lakehouse_table3.select('id', 'ts', 'bodyraw_json'))))))
bodyraw_json_data_customFields = sdf_bodyraw.select(F.explode('bodyraw_json_data_customFields'))
bodyraw_json_data_jobBoards = sdf_bodyraw.select(F.explode('bodyraw_json_data_jobBoards'))
bodyraw_json_data_vacancyCodes = sdf_bodyraw.select(F.explode('bodyraw_json_data_vacancyCodes'))
sdf_bodyraw.printSchema()
table_list=[sdf_bodyraw, bodyraw_json_data_customFields, bodyraw_json_data_jobBoards, bodyraw_json_data_vacancyCodes]
table_list_names=['sdf_bodyraw', 'bodyraw_json_data_customFields', 'bodyraw_json_data_jobBoards', 'bodyraw_json_data_vacancyCodes']

root
 |-- id: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- bodyraw_json__corrupt_record: string (nullable = true)
 |-- bodyraw_json_companyName: string (nullable = true)
 |-- bodyraw_json_date: string (nullable = true)
 |-- bodyraw_json_event: string (nullable = true)
 |-- bodyraw_json_id: string (nullable = true)
 |-- bodyraw_json_data_additionalInformation: string (nullable = true)
 |-- bodyraw_json_data_addressCity: string (nullable = true)
 |-- bodyraw_json_data_addressCountry: string (nullable = true)
 |-- bodyraw_json_data_addressCountryShortName: string (nullable = true)
 |-- bodyraw_json_data_addressLatitude: double (nullable = true)
 |-- bodyraw_json_data_addressLine: string (nullable = true)
 |-- bodyraw_json_data_addressLongitude: double (nullable = true)
 |-- bodyraw_json_data_addressNumber: string (nullable = true)
 |-- bodyraw_json_data_addressState: string (nullable = true)
 |-- bodyraw_json_data_addressStateShortName: string (nullable = true)
 |-- bod

In [19]:
sdf_body = flatten_df((flatten_df(flatten_df(flatten_df(sdf_lakehouse_table3.select('id', 'ts', 'body_json'))))))
body_json_data_customFields = sdf_body.select(F.explode('body_json_data_customFields'))
body_json_data_jobBoards = sdf_body.select(F.explode('body_json_data_jobBoards'))
body_json_data_vacancyCodes = sdf_body.select(F.explode('body_json_data_vacancyCodes'))
sdf_bodyraw.printSchema()

root
 |-- id: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- bodyraw_json__corrupt_record: string (nullable = true)
 |-- bodyraw_json_companyName: string (nullable = true)
 |-- bodyraw_json_date: string (nullable = true)
 |-- bodyraw_json_event: string (nullable = true)
 |-- bodyraw_json_id: string (nullable = true)
 |-- bodyraw_json_data_additionalInformation: string (nullable = true)
 |-- bodyraw_json_data_addressCity: string (nullable = true)
 |-- bodyraw_json_data_addressCountry: string (nullable = true)
 |-- bodyraw_json_data_addressCountryShortName: string (nullable = true)
 |-- bodyraw_json_data_addressLatitude: double (nullable = true)
 |-- bodyraw_json_data_addressLine: string (nullable = true)
 |-- bodyraw_json_data_addressLongitude: double (nullable = true)
 |-- bodyraw_json_data_addressNumber: string (nullable = true)
 |-- bodyraw_json_data_addressState: string (nullable = true)
 |-- bodyraw_json_data_addressStateShortName: string (nullable = true)
 |-- bod

In [20]:
sdf_lakehouse_table3.select('body_json.event', 'body_json.data.application.currentStep.name').distinct().show(truncate=False)

+---------------------+----------------------------+
|event                |name                        |
+---------------------+----------------------------+
|application.created  |Cadastro                    |
|null                 |null                        |
|application.moved    |Interview                   |
|application.moved    |Contratação                 |
|application.completed|Cadastro                    |
|application.moved    |Cadastro                    |
|job.changed          |null                        |
|candidate.hired      |Contratação                 |
|application.moved    |Teste Etapa 3: Entrevista   |
|application.moved    |Teste Etapa 2: Teste Técnico|
+---------------------+----------------------------+



## Pandas Profilling

In [17]:
table_list=[sdf_body, body_json_data_customFields, body_json_data_jobBoards, body_json_data_vacancyCodes]
table_list_names=['sdf_body', 'body_json_data_customFields', 'body_json_data_jobBoards', 'body_json_data_vacancyCodes']

In [34]:
for i in range(len(table_list)):
    df_profile = table_list[i].toPandas()
    if len(df_profile) > 0:
        profile = ProfileReport(df_profile, title=f"Pandas Profiling Report for %s" % table_list_names[i]
                            ,explorative=True
                            ,samples=None
                            ,correlations=None
                            ,missing_diagrams=None
                            ,duplicates=None
                            ,interactions=None
                           )
        profile.to_file("./EDA_datalake/profile_%s.html" % table_list_names[i])

c:\program files\python37\lib\site-packages\matplotlib\textpath.py:84: RuntimeWarning: Glyph 65279 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
c:\program files\python37\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 65279 missing from current font.
  font.set_text(s, 0.0, flags=flags)
c:\program files\python37\lib\site-packages\matplotlib\backends\backend_agg.py:183: RuntimeWarning: Glyph 65279 missing from current font.
  font.set_text(s, 0, flags=flags)


## Using Spark SQL

In [26]:
sdf_lakehouse_table.createOrReplaceTempView("lakehouse_table")

In [28]:
spark.sql("select * from lakehouse_table").show()

+--------------------+--------------------+-------------+------------------+---------------------+------------------+------------------+--------------------+----------------------+-----------------+--------------------+------------------+-----------------------+----------------------+-----------------------+----------------+----------------+----------+--------------------+--------------------+---------------+--------------------+---------------+-------------+--------------------+-------------+---------------+--------------------+---------------+--------+
|              job_id|            job_name|job_seniority|min_salary_offered|target_salary_offered|max_salary_offered|job_specialization|job_primary_language|job_secondary_language|job_contract_type|   job_city_location|job_state_location|flag_housing_assistance|flag_location_required|flag_talent_suggestions|prospects_number|job_hires_number|job_status|           tenant_id|      inhire_user_id|created_at_date| created_at_datetime|create

In [35]:
spark.sql("""
CREATE TEMP VIEW lakehouse_table2 as 
select * 
,case when job_seniority = 'leadership' then 'Management'
when job_seniority in ('junior', 'senior', 'specialist', 'mid-level') then 'Technical'
else 'UNKNOWN' end as job_position
from lakehouse_table
""")
spark.sql("select job_seniority, job_position from lakehouse_table2").show()

+-------------+------------+
|job_seniority|job_position|
+-------------+------------+
|       senior|   Technical|
|   leadership|  Management|
|    mid-level|   Technical|
|       junior|   Technical|
|    mid-level|   Technical|
|    mid-level|   Technical|
|       senior|   Technical|
|   specialist|   Technical|
|       senior|   Technical|
|    mid-level|   Technical|
|       senior|   Technical|
|       senior|   Technical|
|       senior|   Technical|
|    mid-level|   Technical|
|    mid-level|   Technical|
|       senior|   Technical|
|   leadership|  Management|
|    mid-level|   Technical|
|   leadership|  Management|
|    mid-level|   Technical|
+-------------+------------+
only showing top 20 rows



## Import data from Lakehouse

In [5]:
query="""SELECT * FROM "prod-lakehouse-stream"."savedpredict";;"""
df_lakehouse_table = wr.athena.read_sql_query(query, database="prod-lakehouse-mirror", boto3_session=my_boto3_session)
df_lakehouse_table = df_lakehouse_table.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('missing'))
df_lakehouse_table = df_lakehouse_table.fillna(0)
df_lakehouse_table

,job_area,max_salary_offered,import_policy_word_count,job_technical_requirements_word_count,job_validation_questions_word_count,prospect_companies_qty,total_experience_months,experience_duration_months_min,experience_duration_months_clean_avg,experience_duration_months_clean_stddev,...,company_classification_migration,id,job_id,linkedin_user_name,probability,rating,suggestion,image_path,model_id,saved_at_datetime
0,Data,13500.0,6.0,3.0,4.0,1,0,0,0.050000,0.070711,...,Missing-to-Missing,1198-sara-batista-166630176,1198,sara-batista-166630176,0.2626,10,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T01:04:57.067277
1,Data,13500.0,6.0,3.0,4.0,1,0,0,0.050000,0.070711,...,Missing-to-Missing,1198-sara-batista-166630176,1198,sara-batista-166630176,0.2626,10,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T23:50:03.173504
2,Data,13500.0,6.0,3.0,4.0,1,0,0,0.050000,0.070711,...,Missing-to-Missing,1198-sara-batista-166630176,1198,sara-batista-166630176,0.2626,10,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T01:39:07.864316
3,Data,13500.0,6.0,3.0,4.0,5,109,6,18.800000,10.183320,...,High Tech B-to-Missing,1198-thalita-bueno,1198,thalita-bueno,0.0609,3,Repensar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T00:51:27.203370
4,Data,13500.0,6.0,3.0,4.0,1,0,0,0.050000,0.070711,...,Missing-to-Missing,1198-sara-batista-166630176,1198,sara-batista-166630176,0.2626,10,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T23:50:20.003268
5,Data,13500.0,6.0,3.0,4.0,1,0,0,0.050000,0.070711,...,Missing-to-Missing,1198-sara-batista-166630176,1198,sara-batista-166630176,0.2626,10,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T01:07:38.838713
6,Data,13500.0,6.0,3.0,4.0,1,0,0,0.050000,0.070711,...,Missing-to-Missing,1198-sara-batista-166630176,1198,sara-batista-166630176,0.2626,10,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T23:49:49.647887
7,Data,13500.0,6.0,3.0,4.0,1,0,0,0.050000,0.070711,...,Missing-to-Missing,1198-sara-batista-166630176,1198,sara-batista-166630176,0.2626,10,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T01:39:15.645242
8,Data,13500.0,6.0,3.0,4.0,1,0,0,0.050000,0.070711,...,Missing-to-Missing,1198-sara-batista-166630176,1198,sara-batista-166630176,0.2626,10,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T01:07:44.016677
9,Data,13500.0,6.0,3.0,4.0,2,34,6,8.666667,2.516611,...,Empresa Tradicional A-to-Missing,1198-patriciakano,1198,patriciakano,0.1537,8,Abordar,https://vagas-media.s3.amazonaws.com/sherlock/...,GBM_grid_1_AutoML_1_20220602_224459_model_101.zip,2022-06-13T00:43:45.985840


In [6]:
# sdf_lakehouse_table = spark.createDataFrame(linkedincompanies, schema)
sdf_lakehouse_table = spark.createDataFrame(df_lakehouse_table)
sdf_lakehouse_table.printSchema()

root
 |-- job_area: string (nullable = true)
 |-- max_salary_offered: double (nullable = true)
 |-- import_policy_word_count: double (nullable = true)
 |-- job_technical_requirements_word_count: double (nullable = true)
 |-- job_validation_questions_word_count: double (nullable = true)
 |-- prospect_companies_qty: long (nullable = true)
 |-- total_experience_months: long (nullable = true)
 |-- experience_duration_months_min: long (nullable = true)
 |-- experience_duration_months_clean_avg: double (nullable = true)
 |-- experience_duration_months_clean_stddev: double (nullable = true)
 |-- last_experience_duration_months_to_avg: double (nullable = true)
 |-- last_experience_descriptions_word_count: double (nullable = true)
 |-- last_company_classification: string (nullable = true)
 |-- all_company_classifications_count: long (nullable = true)
 |-- declared_seniority: string (nullable = true)
 |-- declared_seniority_migration: string (nullable = true)
 |-- prospect_location_state: string